In [38]:
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

import tensorflow

from scikeras.wrappers import KerasClassifier
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
import pickle

In [2]:
data = pd.read_csv(r'C:\Users\deepa\Panda 2024\GenAI\Churn Prediction\dataset\Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
os.getcwd()

'c:\\Users\\deepa\\Panda 2024\\GenAI\\Churn Prediction'

In [4]:
data = data.drop(["RowNumber","CustomerId","Surname"],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
label_encoder_gender_for_regression = LabelEncoder()
data['Gender']=label_encoder_gender_for_regression.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [10]:
one_hot_encoder_for_regression = OneHotEncoder(sparse_output=True)
geography_for_regreesion_df = one_hot_encoder_for_regression.fit_transform(data[['Geography']]).toarray()
geography_for_regreesion_df_with_columnname = pd.DataFrame(geography_for_regreesion_df,columns=one_hot_encoder_for_regression.get_feature_names_out(['Geography']))

In [11]:
geography_for_regreesion_df_with_columnname

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [15]:
df = pd.concat([data.drop("Geography",axis=1),geography_for_regreesion_df_with_columnname],axis=1)
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [16]:
X = df.drop('Exited',axis=1)
y = df['Exited']

In [18]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [20]:
X_train

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
9254,686,1,32,6,0.00,2,1,1,179093.26,1.0,0.0,0.0
1561,632,1,42,4,119624.60,2,1,1,195978.86,0.0,1.0,0.0
1670,559,1,24,3,114739.92,1,1,0,85891.02,0.0,0.0,1.0
6087,561,0,27,9,135637.00,1,1,0,153080.40,1.0,0.0,0.0
6669,517,1,56,9,142147.32,1,0,0,39488.04,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,768,1,54,8,69712.74,1,1,1,69381.05,1.0,0.0,0.0
5191,682,0,58,1,0.00,1,1,1,706.50,1.0,0.0,0.0
5390,735,0,38,1,0.00,3,0,0,92220.12,1.0,0.0,0.0
860,667,1,43,8,190227.46,1,1,0,97508.04,1.0,0.0,0.0


In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
X_train.shape

(8000, 12)

In [26]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [24]:
X_test.shape

(2000, 12)

In [25]:
with open("reg_label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender_for_regression,file)

with open("reg_one_hot_encoder_geo.pkl","wb") as file:
    pickle.dump(one_hot_encoder_for_regression,file)

with open("reg_scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

# ANN Training

In [40]:
def model_train(neuron = 32, hidden_layer = 1):
    model = Sequential()
    model.add(Dense(neuron,activation="relu",input_shape=(X_train.shape[1],)))

    for i in range(hidden_layer-1):
        model.add(Dense(neuron,activation="relu"))

    model.add(Dense(1,activation="sigmoid"))

    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

    return model


In [41]:
params = {
    'neuron':[16,32,64,128],
    'hidden_layer':[1,2],
    'epochs' : [50,100]
}

In [42]:
model = KerasClassifier(neuron = 32, hidden_layer = 1,build_fn=model_train,verbose=1)

In [43]:
grid_search = GridSearchCV(estimator=model,param_grid=params,cv=3,verbose=1)

grid_result = grid_search.fit(X_train,y_train)

print(f"Best score: {grid_result.best_score_}")
print(f"Best param: {grid_result.best_params_}")

Fitting 3 folds for each of 16 candidates, totalling 48 fits


c:\Users\deepa\Panda 2024\GenAI\Churn Prediction\churn_env\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)



Epoch 1/50


167/167 [==============================] - 3s 2ms/step - loss: 0.8207 - accuracy: 0.4849
Epoch 2/50
167/167 [==============================] - 0s 2ms/step - loss: 0.4958 - accuracy: 0.7919
Epoch 3/50
167/167 [==============================] - 0s 2ms/step - loss: 0.4473 - accuracy: 0.8007
Epoch 4/50
167/167 [==============================] - 0s 2ms/step - loss: 0.4310 - accuracy: 0.8076
Epoch 5/50
167/167 [==============================] - 0s 2ms/step - loss: 0.4205 - accuracy: 0.8134
Epoch 6/50
167/167 [==============================] - 0s 2ms/step - loss: 0.4121 - accuracy: 0.8206
Epoch 7/50
167/167 [==============================] - 0s 2ms/step - loss: 0.4044 - accuracy: 0.8258
Epoch 8/50
167/167 [==============================] - 0s 2ms/step - loss: 0.3971 - accuracy: 0.8333
Epoch 9/50
167/167 [==============================] - 0s 2ms/step - loss: 0.3903 - accuracy: 0.8361
Epoch 10/50
167/167 [==============================] - 0s 2ms/step - loss: 0.3838 - accuracy: 0.8

c:\Users\deepa\Panda 2024\GenAI\Churn Prediction\churn_env\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


167/167 [==============================] - 1s 2ms/step - loss: 0.5646 - accuracy: 0.7238
Epoch 2/50
167/167 [==============================] - 0s 2ms/step - loss: 0.4624 - accuracy: 0.8059
Epoch 3/50
167/167 [==============================] - 0s 2ms/step - loss: 0.4372 - accuracy: 0.8136
Epoch 4/50
167/167 [==============================] - 0s 2ms/step - loss: 0.4249 - accuracy: 0.8183
Epoch 5/50
167/167 [==============================] - 0s 2ms/step - loss: 0.4156 - accuracy: 0.8234
Epoch 6/50
167/167 [==============================] - 0s 2ms/step - loss: 0.4064 - accuracy: 0.8271
Epoch 7/50
167/167 [==============================] - 0s 2ms/step - loss: 0.3983 - accuracy: 0.8324
Epoch 8/50
167/167 [==============================] - 0s 2ms/step - loss: 0.3905 - accuracy: 0.8378
Epoch 9/50
167/167 [==============================] - 0s 2ms/step - loss: 0.3835 - accuracy: 0.8395
Epoch 10/50
167/167 [==============================] - 0s 2ms/step - loss: 0.3775 - accuracy: 0.8431
Epoch 11/5

...

167/167 [==============================] - 1s 1ms/step - loss: 0.4470 - accuracy: 0.8117
Epoch 2/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3698 - accuracy: 0.8491
Epoch 3/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3468 - accuracy: 0.8571
Epoch 4/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3410 - accuracy: 0.8601
Epoch 5/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3353 - accuracy: 0.8626
Epoch 6/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3330 - accuracy: 0.8642
Epoch 7/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3295 - accuracy: 0.8626
Epoch 8/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3225 - accuracy: 0.8650
Epoch 9/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3201 - accuracy: 0.8680
Epoch 10/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3163 - accuracy: 0.8657
E

c:\Users\deepa\Panda 2024\GenAI\Churn Prediction\churn_env\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


167/167 [==============================] - 1s 1ms/step - loss: 0.4355 - accuracy: 0.8136
Epoch 2/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3666 - accuracy: 0.8528
Epoch 3/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3433 - accuracy: 0.8598
Epoch 4/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3357 - accuracy: 0.8637
Epoch 5/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3285 - accuracy: 0.8660
Epoch 6/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3256 - accuracy: 0.8660
Epoch 7/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3215 - accuracy: 0.8686
Epoch 8/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3180 - accuracy: 0.8680
Epoch 9/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3128 - accuracy: 0.8682
Epoch 10/100
167/167 [==============================] - 0s 1ms/step - loss: 0.3110 - accuracy: 0.8723
E

c:\Users\deepa\Panda 2024\GenAI\Churn Prediction\churn_env\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


250/250 [==============================] - 1s 955us/step - loss: 0.5160 - accuracy: 0.7642
Epoch 2/50
250/250 [==============================] - 0s 985us/step - loss: 0.4260 - accuracy: 0.8206
Epoch 3/50
250/250 [==============================] - 0s 981us/step - loss: 0.4087 - accuracy: 0.8261
Epoch 4/50
250/250 [==============================] - 0s 939us/step - loss: 0.3937 - accuracy: 0.8364
Epoch 5/50
250/250 [==============================] - 0s 931us/step - loss: 0.3775 - accuracy: 0.8462
Epoch 6/50
250/250 [==============================] - 0s 942us/step - loss: 0.3649 - accuracy: 0.8521
Epoch 7/50
250/250 [==============================] - 0s 955us/step - loss: 0.3550 - accuracy: 0.8545
Epoch 8/50
250/250 [==============================] - 0s 929us/step - loss: 0.3489 - accuracy: 0.8571
Epoch 9/50
250/250 [==============================] - 0s 939us/step - loss: 0.3451 - accuracy: 0.8585
Epoch 10/50
250/250 [==============================] - 0s 939us/step - loss: 0.3417 - accurac

In [45]:
with open("reg_model.pkl","wb") as file:
    pickle.dump(model,file)